In [ ]:
!ls real_data/

In [36]:
import os
import pandas as pd
import numpy as np
import progressbar
import time
import q4functions as q4funcs


def predict(train,test,n_ap):
    pass


def get_training_data(path,calculate_means_and_variances=True):
    ''' Loads in training data from path argument, and returns in matrix for with mac addresses as columns.
        If calculate_means_and_variances=True the entries are the means and standard deviations of the readings,
        else just a list of all samples is returned
    '''
    print(f'Loading training data from {path}')
    time.sleep(0.2)
    locations_df = pd.read_csv(os.path.join(path,'location.txt'),header=None)
    locations_df.columns = ['x','y']
    features_path = os.path.join(path,'wifi_signal')
    feature_csvs = os.listdir(features_path)
    train_df = pd.DataFrame()
    for feature_csv in progressbar.progressbar(feature_csvs,prefix=f'Loading training data from {path}'):
        location = np.array(locations_df.iloc[int(feature_csv[6:].split('.')[0])-1])
#         corresponding_location = locations_df[]
        feature_df = pd.read_csv(os.path.join(features_path,feature_csv),header=None)
        feature_df.columns = ['timestamp','router_name','mac_address','channel','RSSI']
        unique_mac_addresses = list(set(feature_df['mac_address']))
        location_dict = {'x': location[0], 'y': location[1]}
        
        for unique_mac_address in unique_mac_addresses:
            relevant_entries = feature_df.iloc[np.where(feature_df['mac_address']==unique_mac_address)]
            rssi_values = relevant_entries['RSSI'] + 0.5*np.random.random(size=len(relevant_entries))
            if calculate_means_and_variances:
                #             add artificial noise to avoid 0 standard deviation
                mean = rssi_values.mean()
                std_dv = rssi_values.std()
                means_and_stdevs = [mean,std_dv]
                location_dict[unique_mac_address] = means_and_stdevs 
            else:
                location_dict[unique_mac_address] = rssi_values
        train_df =train_df.append(location_dict,ignore_index=True)
    train_df = train_df.reindex(columns = ['x','y'] + [col for col in train_df.columns if col not in ['x','y'] ])
    return train_df

def get_test_data(path):
    locations_df = pd.read_csv(os.path.join(path,'location.txt'),header=None)
    features_path = os.path.join(path,'wifi_signal')
    feature_csvs = os.listdir(features_path)
    test_df = pd.DataFrame()
    for feature_csv in feature_csvs:

        feature_df = pd.read_csv(os.path.join(features_path,feature_csv))
        feature_df.columns = ['timestamp','router_name','mac_address','channel','RSSI']
        unique_mac_addresses = list(set(feature_df['mac_address']))
        location_dict = {'x': 0, 'y': 0}
        
        for unique_mac_address in unique_mac_addresses:
            relevant_entries = feature_df.iloc[np.where(feature_df['mac_address']==unique_mac_address)]
            rssi_values = relevant_entries['RSSI']
            location_dict[unique_mac_address] = rssi_values
        test_df =test_df.append(location_dict,ignore_index=True)
    test_df = test_df.reindex(columns = ['x','y'] + [col for col in test_df.columns if col not in ['x','y'] ])
    return test_df

def make_predictions(train_data,test_data):
    locations = []
    for i in range(len(test_data)):
        test_features = pd.DataFrame(test_data.iloc[i]).transpose()
        test_mac_addresses = set(test_features.columns[2:])
        train_mac_addresses = set(train_data.columns[2:])
        
        
        mutual_mac_addresses = list(test_mac_addresses.intersection(train_mac_addresses))
        cropped_test_df = test_features[['x','y']+ mutual_mac_addresses]
        
        cropped_test_df = pd.DataFrame(cropped_test_df.apply(lambda x: np.mean(x[0]))).transpose()
        cropped_train_df = train_data[['x','y']+mutual_mac_addresses]
        location = q4funcs.predict(train_data,test_data,len(mutual_mac_addresses))
        locations.append(location)
        print(location)
        break
    return cropped_test_df,cropped_train_df

def preprocess_training_data(*args):
    pass
    
training_path = "./real_data/training/"
test_path = "./real_data/test/"

# get_training_data(training_path,calculate_means_and_variances=True).head()
test_data = get_test_data(test_path)
train_data = get_training_data(training_path)

a,b = make_predictions(train_data,test_data)


Loading training data from ./real_data/training/


Loading training data from ./real_data/training/100% (15 of 15) || Elapsed Time: 0:00:00 Time:  0:00:00


TypeError: predict() missing 1 required positional argument: 'samples_array'

In [32]:
a.head()
# b.head()

,x,y,00:2a:10:3d:b0:1e,10:9a:dd:a0:fb:1e,18:8b:45:09:70:e0,00:81:c4:85:07:a1,40:18:b1:78:89:d5,40:18:b1:78:a9:55,18:8b:45:09:70:e1,00:2a:10:3d:b0:11,...,08:ea:44:96:ec:96,08:ea:44:96:ec:eb,18:8b:45:09:70:ef,40:18:b1:7b:42:14,e8:de:27:6f:f5:d0,00:2a:10:35:39:31,00:2a:10:1a:69:80,18:8b:45:09:72:61,00:2a:10:3d:b0:10,e8:de:27:6f:f5:d1
0,0.0,0.0,-73.823529,-84.848485,-79.961538,-45.029412,-86.296296,-91.0,-80.769231,-68.411765,...,-87.5,-84.647059,-83.103448,-87.055556,-45.029412,-85.0,-86.8,NaN,-68.441176,-41.647059


In [35]:
b.head()

,x,y,00:2a:10:3d:b0:1e,10:9a:dd:a0:fb:1e,18:8b:45:09:70:e0,00:81:c4:85:07:a1,40:18:b1:78:89:d5,40:18:b1:78:a9:55,18:8b:45:09:70:e1,00:2a:10:3d:b0:11,...,08:ea:44:96:ec:96,08:ea:44:96:ec:eb,18:8b:45:09:70:ef,40:18:b1:7b:42:14,e8:de:27:6f:f5:d0,00:2a:10:35:39:31,00:2a:10:1a:69:80,18:8b:45:09:72:61,00:2a:10:3d:b0:10,e8:de:27:6f:f5:d1
0,1.35,6.90,"[-75.10175629407068, 1.4266113663500397]","[-86.78276919591832, 0.8755223009245243]","[-76.18123975272594, 3.3991426868904133]","[-65.16474895188722, 2.6871210952558933]","[-84.0834839039512, 2.0157034509077936]","[-83.72567744056822, 0.12864559957557134]","[-76.10162557063231, 3.369872939940462]","[-64.49925225055442, 0.980863273043145]",...,"[-88.73513654282945, 1.5591341264061036]","[-87.02182314182518, 1.001636012448375]","[-83.0628585923047, 0.85267560155798]","[-88.05920516171756, 1.6840420066988118]","[-43.63235019878584, 2.0033480079223964]","[-82.74117380588666, 0.19706195970492438]","[-81.74240533870896, 0.11409103279438745]",NaN,"[-63.76561919087961, 0.954173650741619]","[-37.40505188238275, 1.0592350690841699]"
1,7.95,8.20,"[-74.00826288692517, 2.684512908925793]","[-81.39735751007709, 3.143025390057527]","[-78.42676174927563, 1.26892818827496]","[-58.93195619044049, 3.6672179903880204]","[-87.11415042104616, 2.504606298239307]","[-90.73129265309674, 0.20000492198097772]","[-78.27012081597039, 1.5555866243872727]","[-68.2778167930453, 2.3149137136940436]",...,NaN,"[-87.85693740328841, 1.769058598438946]","[-82.85203679464459, 1.136243626408617]","[-89.79339008483414, 0.16167997873022932]","[-46.786060190298855, 1.7331573784752006]",NaN,NaN,"[-77.24212515361656, 1.4992439691947845]","[-69.53456257138758, 2.6866330174765625]","[-42.05234985966788, 2.594751435243722]"
2,7.95,6.67,"[-74.81197678633617, 1.930725314157962]","[-84.01282561594772, 1.7322049918094977]","[-77.931952185281, 1.9728347246248747]","[-59.6002644084167, 3.3425274980460538]","[-86.75829461119949, 1.7634695644188494]","[-91.6923129311495, 0.15475415425549585]","[-77.63224904592596, 2.013825968074519]","[-68.11239741735724, 2.1875345741200745]",...,"[-88.75813372775876, 0.1348979430782003]","[-84.74323377159652, 1.6335567517990073]","[-83.02342894159396, 1.2400422173032637]","[-89.89458150867003, 0.7526383277149622]","[-45.53610649695917, 1.8453207122977389]",NaN,NaN,"[-77.62540924993803, 2.329381987863178]","[-69.72184966744143, 3.2706662901437724]","[-43.432384698998455, 3.3065062468630564]"
3,1.35,8.80,"[-77.88151395082052, 2.0226904200192153]","[-86.6661361499485, 2.0254174743446844]","[-77.59840982109228, 2.486233842403664]","[-61.29563845154135, 2.841479231339811]","[-87.82486267684823, 1.4970132813908572]",NaN,"[-76.41128022631882, 2.3195765879742276]","[-74.41953822195217, 4.922936191340135]",...,"[-90.22446581205733, 0.4292737628764254]","[-87.6657511359801, 0.39162964894485297]","[-82.97601066600097, 1.3411578519668808]",NaN,"[-45.628455144274305, 1.8427643491685095]",NaN,NaN,"[-83.73157400037479, 0.12792167413258546]","[-74.28583592449397, 3.4941641562238086]","[-36.887902307227534, 1.9263686647351306]"
4,4.70,2.80,"[-69.57438932813687, 2.5276012553716773]","[-86.39799496215066, 1.4356860776467701]","[-79.69795625323714, 0.1568612978985201]","[-52.07870557372391, 1.3033307050371383]","[-84.84509529740781, 1.3174029597970844]","[-88.86475289634697, 1.1302350717540863]","[-81.16168847274436, 0.63071215562503]","[-67.25832443999653, 2.7695857592689883]",...,"[-84.78660903640639, 2.0065287377690995]","[-83.61725218396761, 1.593539372184322]","[-82.71963247806043, 1.0399787485340577]","[-88.51521007232344, 1.2374534071919425]","[-46.183580509354194, 1.9359822274062548]",NaN,"[-84.8994570979783, 0.37578549050325044]",NaN,"[-66.07555063594683, 3.6224149443010134]","[-43.42531760302412, 3.351938286579725]"
